# Physical Space and the Image Grid

### Learning Objectives

* Understand how an image pixel grid is defined in physical space
* Understand how a spatial transform maps two physical spaces
* Understand how to define a resampling operation to generate a new image grid

## Overview

## Exercise 1: 

## Exercise 2: